In [1]:
import numpy as np
import pandas as pd
from Func import *
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.__version__
pd.__version__

'1.3.5'

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
1,US as Home,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
2,EU28 as Home,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
3,OECD37 as Home,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
4,World as Home,32.175999,0.100000,32.276001,32.146000,0.030000,0.030000,0.070000,32.175999,0.100000,32.175999,0.100000,32.276001,32.276001,0.300000,0.999068
5,China as Home,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
6,OECD and China as Home,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
7,US and EU as Home,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [4]:
## choose which regional scenario to run (runs all if not executed)
    #df=df.drop([1,2,4,5,6,7])  

In [5]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [6]:
tax_scenario= pd.DataFrame({'tax_sce': ['Baseline','Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[1,0,1,1,1,1,1,1,1]},index=[1, 2, 3, 4, 5, 6, 7, 8, 9])
# use for quick test: tax_scenario= pd.DataFrame({'tax_sce': 'EPC_hybrid', 'Base':1},index=[1])

In [7]:
def cases(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    # varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.arange (0,20.1,0.2) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = [2]
    #varphilist = np.arange (1.7,2.5,0.1)
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(minimization, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
        output.append(output_df)
        print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb

    print(tax_scenario['tax_sce'])
    return output


In [8]:
## apply the above for each tax scenario
output_all=tax_scenario.apply(cases, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

2
Baseline
tax returned is [  0.7266707 -71.7681086] flag is True tax scenario is Unilateral
tax returned is [  0.70722825 -69.6782938 ] flag is True tax scenario is Unilateral
tax returned is [  0.84650591 -83.66348152] flag is True tax scenario is Unilateral
tax returned is [  0.98911279 -97.9362793 ] flag is True tax scenario is Unilateral
tax returned is [  0.7741117  -76.40491982] flag is True tax scenario is Unilateral
tax returned is [  0.98566837 -97.59201355] flag is True tax scenario is Unilateral
tax returned is [  0.77888603 -76.81262674] flag is True tax scenario is Unilateral
2
Unilateral
2
purete
tax returned is [  0.7197384  -70.99884033] flag is True tax scenario is puretc
tax returned is [  0.70175145 -69.20248885] flag is True tax scenario is puretc
tax returned is [  0.81697968 -80.64796753] flag is True tax scenario is puretc
pe is (array([1.]), {'nfev': 13, 'fjac': array([[-1.]]), 'r': array([nan]), 'qtf': array([-21.61870264]), 'fvec': array([21.61870264])}, 5, '

In [ ]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])

In [13]:
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)

In [19]:
Outcomes.to_csv('../output/temp_alex.csv'.format(case), header=True) 

In [ ]:
if epsilonS == 0.5 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}.csv'.format(case), header=True) 
elif epsilonS == 0.5 and epsilonSstar == 2:
    Outcomes.to_csv('output/output_case{0}_D_2.csv'.format(case), header=True) 
elif epsilonS == 2 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}_2_D.csv'.format(case), header=True) 

In [21]:
!pip install palettable

     -------------------------------------- 111.8/111.8 kB 2.2 MB/s eta 0:00:00
